# Evaluation ?

In [45]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


In [46]:
%%capture
%run 4.Modeling.ipynb

In [47]:
# model_list [0] == liniar regesion model
#  model_list[1] == randomforest
#  model_list[2] == target encoder
model_list = return_models() 


In [48]:
table = pd.read_csv('Validatieset_2020_2021_CSV.csv', sep=';')

In [49]:
def get_prediction_forest(model_list, table):
    prediction_list = []
    for item in table.values:
        prediction_list.append(model_list[1].predict([item]))
        
        # Algemene RSME =55 minuten,  r2=0.79  ( random forest )
        
    return prediction_list

In [50]:
def get_prediction_linreg(model_list, table):
    prediction_list = []
    for item in table.values:
        prediction_list.append(model_list[0].predict([item]))

        # Algemene RSME =108 minuten,  r2=0.21  ( random linreg )
        
    return prediction_list

In [51]:
def change_target_encoder(encoder, table):
    table['stm_equipm_soort_mld_looe_mean'] = encoder.transform(table['stm_equipm_soort_mld'].astype('category'))
    return table

In [52]:
def change_type_create_duration(table):
    table['stm_aanntpl_ddt'] = pd.to_datetime(table['stm_aanntpl_ddt'], errors='coerce', dayfirst=False, infer_datetime_format=True)
    table['stm_aanngeb_ddt'] = pd.to_datetime(table['stm_aanngeb_ddt'], errors='coerce', dayfirst=False, infer_datetime_format=True)
    table['stm_gebeld_ter_plekken_duur'] = (table['stm_aanntpl_ddt'] - table['stm_aanngeb_ddt']).astype('timedelta64[m]')
    return table
    

In [ ]:
# --------------Dit hier onder zijn de voorbeelden van de functies die gedraait moeten worden-----------------------

In [54]:
#  Er moeten kolomen aan gemaakt worden zo als duur tusen tijd en target encoding  
table = change_type_create_duration(table)
table = change_target_encoder(model_list[2], table)
# hier boven staan de funcite, mee geven wordt op moment validatie zet en de model[2] encoder

In [55]:
# # Voorbeeld oproepen prediction
print(get_prediction_linreg(model_list, table[['stm_progfh_in_duur', 'stm_equipm_soort_mld_looe_mean']]))

# # Voorbeeld oproepen prediction
print(get_prediction_forest(model_list, table[['stm_equipm_soort_mld_looe_mean', 'stm_progfh_in_duur', 'stm_gebeld_ter_plekken_duur']]))


#  Hier onder staan de validatie zet antwoorden



[array([111.36908559]), array([114.32367913]), array([192.59791927])]
[array([67.5072604]), array([111.82794123]), array([138.61162732])]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=df0a20df-15ef-4481-b361-742b03a94b10' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>